In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
import openpyxl
import csv
from bs4 import BeautifulSoup

base_url = "https://www.amazon.in/s?k=bags&crid=2M096C61O4MLT&qid=1653308124&sprefix=ba%2Caps%2C283&ref=sr_pg_"
pages = 20

for i in range(1, pages + 1):

    # Create URL to collect links from paginated data
    url = f"{base_url}{i}"

    # Collect HTML data from this page
    response = requests.get(url)
   # print(response)
# Amazon is throwing 503 error, as such I am downloading first page to show as example on how to extract data from it.
# Read the HTML content from the file
with open('AmazonPage1.htm', 'r', encoding='utf-8') as file:
    html_content = file.read()

# Parse the HTML content with BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

# Find all product containers
product_containers = soup.find_all('div', class_='s-card-container')

# Create a new Excel workbook and add a worksheet
workbook = openpyxl.Workbook()
worksheet = workbook.active
worksheet.title ='Amazon Product List'

# Write headers to the worksheet
headers = ["Product Url",'Product Name', 'Product Price', 'Product Rating', 'Product Reviews']
worksheet.append(headers)



# Extract and write product information to the worksheet
for container in product_containers:
    product_url = container.find('a', class_='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal').get('href', '')
    product_name = container.find('span', class_='a-size-medium a-color-base a-text-normal').text.strip() if container.find('span', class_='a-size-medium a-color-base a-text-normal') else ''
    product_price = container.find('span', class_='a-price-whole').text if container.find('span', class_='a-price-whole') else ''
    product_rating = container.find('span', class_='a-size-base puis-normal-weight-text').text if container.find('span', class_='a-size-base puis-normal-weight-text') else ''
    product_reviews = container.find('span', class_='a-size-base a-color-secondary totalRatingCount').text if container.find('span', class_='a-size-base a-color-secondary totalRatingCount') else ''
# Product Reviews is not available on a static page. Hence these values are empty
    
    # Write the data to the worksheet
    worksheet.append([product_url, product_name, product_price, product_rating, product_reviews])

In [3]:
for row in worksheet.iter_rows(min_row=2, max_row=worksheet.max_row, min_col=1, max_col=1):
    cell_value = row[0].value
    response = requests.get(cell_value)
print(response)
# Amazon is throwing 503 error, as such I am downloading product page to show as example on how to extract data from it.

<Response [503]>


In [4]:
with open('ProductPage.htm', 'r', encoding='utf-8') as file:
    html_content = file.read()

In [5]:
product=workbook.create_sheet('Product Details')

In [6]:
headers = ["Product Title","Description",'Asin', 'Manufacturer']
product.append(headers)

In [7]:
# Parse the HTML content with BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

In [8]:
product_title=soup.find('span', class_="a-size-large product-title-word-break").get_text() if soup.find('span', class_="a-size-large product-title-word-break") else ''


In [9]:
manufacturer = soup.find('ul', class_='a-unordered-list a-nostyle a-vertical a-spacing-none detail-bullet-list') \
                      .find_all('span')[8].get_text()


In [10]:
asin=soup.find('ul', class_='a-unordered-list a-nostyle a-vertical a-spacing-none detail-bullet-list') \
                      .find_all('span')[11].get_text()

In [11]:
ul_element = soup.find('ul', class_='a-unordered-list a-vertical a-spacing-mini')
if ul_element:
    span_elements = ul_element.find_all('span')
    description = ' '.join(span.get_text(strip=True) for span in span_elements)


In [12]:
product.append([product_title,description,asin,manufacturer])

In [13]:

# Save the workbook
workbook.save('Amazon_Products.xlsx')

In [14]:
read_file = pd.read_excel (r'Amazon_Products.xlsx')
read_file.to_csv (r'Amazon_Products.csv', index = None, header=True)